In [1]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
import gc
import shutil

# try to reduce unwanted Tensorflow warnings)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR) # or logging.INFO, logging.WARNING, etc.

%load_ext autoreload
%autoreload 1
%aimport ML_builders

!pip install wandb
import wandb
from wandb.integration.keras import WandbCallback

# Set data paths for training and validation sets (change if they are somewhere else)
TRAIN_DATADIR = 'trainvaldata/train'
VAL_DATADIR =  'trainvaldata/val'

# change as you see fit
RESULTS_DIR = 'Results'
if not os.path.exists(RESULTS_DIR):
    os.mkdir(RESULTS_DIR)    

        

In [2]:
learning_params = ML_builders.SetLearningParams()

# Create data loaders

train_ds = tf.keras.utils.image_dataset_from_directory(TRAIN_DATADIR,image_size=(64, 64),batch_size=learning_params['batch_size'])
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

val_ds = tf.keras.utils.image_dataset_from_directory(VAL_DATADIR,image_size=(64, 64),batch_size=learning_params['batch_size'])
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)



Found 98015 files belonging to 41 classes.
Found 32661 files belonging to 41 classes.


In [3]:
# Create and compile model
model_type = 'student_model'
model_name = f"{model_type}-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

my_conv_model = ML_builders.CreateModel(modelname = model_name)

my_conv_model.compile(optimizer=learning_params['optimizer'],
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=["accuracy","sparse_top_k_categorical_accuracy"])
    
my_conv_model.summary()

Model: "student_model-20241114-205118"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling (Rescaling)                │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 64, 64, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 32, 32, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 32, 32, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 16, 16, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 16, 16, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 8, 8, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 4, 4, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 1,184,713 (4.52 MB)

 Trainable params: 1,183,753 (4.52 MB)

 Non-trainable params: 960 (3.75 KB)

In [4]:
# Uncomment and fill in your WandB key here if you want to use WandB
#wandb.login(key= ... )

# Set WandB configuration, containing everything you want to log for this model run
# we advise you to use informative model names and log all relevant model properties and parameters to allow you to compare models later afterwards

#CONFIG = dict (
#
#)
# initialise W&B session here
#run = wandb.init(project='ML2024_DLproject', 
#                 config=CONFIG,
#                 name=model_name,
#                 )

# Train model

tf.random.set_seed(learning_params['student_seed'])
gc.collect()

history = my_conv_model.fit(train_ds,validation_data=val_ds, 
                    callbacks = learning_params['callbacks'],
                    epochs=learning_params['epochs'], verbose = 1)

# Evaluate the model on the test dataset
print('\n\nEvaluating patch model on training set:')
Final_train_results = my_conv_model.evaluate(train_ds)
print('Evaluating patch model on validation set:')
Final_val_results = my_conv_model.evaluate(val_ds)

# NOTE: it may be useful to log the results of these evaluations to W&B as well ...

wandb.finish()

model_savepath = RESULTS_DIR+'/'+model_name
os.mkdir(model_savepath)    

# Save submission files
model_file = os.path.join(model_savepath,f'{model_name}.keras')
my_conv_model.save(model_file)

# save training history
hist_df = pd.DataFrame(history.history) 
hist_csv_file = os.path.join(model_savepath,f'{model_name}_history.csv')
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
 
# copy current ML_builders file to model results folder
shutil.copy2('ML_builders.py',model_savepath)


Epoch 1/20
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 1042s 338ms/step - accuracy: 0.1812 - loss: 3.1472 - sparse_top_k_categorical_accuracy: 0.4691 - val_accuracy: 0.2156 - val_loss: 2.9143 - val_sparse_top_k_categorical_accuracy: 0.5424
Epoch 2/20
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 870s 284ms/step - accuracy: 0.2847 - loss: 2.5962 - sparse_top_k_categorical_accuracy: 0.6318 - val_accuracy: 0.3424 - val_loss: 2.3873 - val_sparse_top_k_categorical_accuracy: 0.6894
Epoch 3/20
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 863s 282ms/step - accuracy: 0.3517 - loss: 2.3168 - sparse_top_k_categorical_accuracy: 0.7082 - val_accuracy: 0.3797 - val_loss: 2.2379 - val_sparse_top_k_categorical_accuracy: 0.7287
Epoch 4/20
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 862s 282ms/step - accuracy: 0.4032 - loss: 2.1161 - sparse_top_k_categorical_accuracy: 0.7529 - val_accuracy: 0.4140 - val_loss: 2.1003 - val_sparse_top_k_categorical_accuracy: 0.7541
Epoch 5/20
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 866s 283ms/step - accuracy: 0.4438 - loss: 1.9619 - s

'Results/student_model-20241114-205118\\ML_builders.py'